In [0]:
i
mport tensorflow as tf
import numpy as np
import os
from google.colab import drive
import matplotlib.pyplot as plt
from scipy.misc import bytescale
!pip3 install -q progressbar2
from progressbar import ProgressBar

import keras
from keras.layers import Input # to make input layer
from keras.models import Model, Sequential # construct models 
from keras.layers.core import Dense, Dropout  # fully-connected = Dense 
from keras.layers.advanced_activations import LeakyReLU 
from keras.optimizers import Adam 
from keras import initializers

In [0]:
drive.mount('/content/drive') # mount drive

In [0]:
# go to my class folder where this code is
os.chdir('drive/My Drive/DeepLearningFall2018') 

In [0]:
batch_size = 100  
n_iters = 100000  # number of training iterations 
noise_size = 100  # how many random numbers to send generator

In [0]:
!pip3 install -q tflearn
import tflearn.datasets.mnist as mnist
X, _, _, _ = mnist.load_data()

In [0]:
# print shape of X and its min and max value
print(X.shape, np.amin(X), np.amax(X))

In [0]:
plt.imshow(X[0, ...].reshape([28, 28]))

In [0]:
X = (X - 0.5) / .5  # make min value -1 and max 1 in X

In [0]:
print(np.amin(X), np.amax(X))

In [0]:
tf.reset_default_graph() # resets the network

In [0]:
def get_generator(optimizer):
  # this function is going to describe the generator network's operations
  generator = Sequential() # create a model where we will stack layers later
  
  generator.add(Dense(256, # 256 nodes going to examine noise and make responses 
                      input_dim=noise_size, # number of inputs to this is 100
                      # initialize the weights foro this layer with random values
                      kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  generator.add(LeakyReLU(0.2))
  
  # another fully-connected layer with 512 nodes to examine 256 previous responses
  generator.add(Dense(512))
  generator.add(LeakyReLU(0.2))
  
  # another fully-connected
  generator.add(Dense(1024))
  generator.add(LeakyReLU(0.2))
  
  # output layer that is the fake image
  generator.add(Dense(784, activation='tanh'))
  
  # tell the generator how to measure its loss
  generator.compile(loss='binary_crossentropy', optimizer=optimizer)
  
  return generator

In [0]:
def get_discriminator(optimizer):
  # this function describes the discriminator network of the GAN
  discriminator = Sequential()  # create an empty model to stack layers in
  
  # first fully-connected layer 
  discriminator.add(Dense(1024, # 1024 nodes going to examine the input image
                          input_dim=784, # 784 nodes in input image
                          # initialize weights of these nodes with random values
                          kernel_initializer=initializers.RandomNormal(stddev=0.02)))
  # activation function
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3)) # drop out 30% of nodes randomly 
  
  # second fully-connected layer
  discriminator.add(Dense(512))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  
  # third fully-connected layer
  discriminator.add(Dense(256))
  discriminator.add(LeakyReLU(0.2))
  discriminator.add(Dropout(0.3))
  
  # output layer
  discriminator.add(Dense(1, activation='sigmoid'))
  
  discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
  
  return discriminator